In [4]:
import torch
import torch.nn.functional as F
import matplotlib

%matplotlib inline

In [6]:
words = open('names.txt', 'r').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}


In [24]:
context_size = 3

xs = []
ys = []
for word in words[:1]:
    context = [0] * context_size
    for ch in word:
        ich = stoi[ch]
        xs.append(context)
        ys.append(ich)
        context = context[1:] + [ich]
    xs.append(context)
    ys.append(0)
assert len(xs) == len(ys)
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs)
print(ys)

tensor([[ 0,  0,  0],
        [ 0,  0,  5],
        [ 0,  5, 13],
        [ 5, 13, 13],
        [13, 13,  1]])
tensor([ 5, 13, 13,  1,  0])


In [ ]:

num_examples = ys.shape[0]

g = torch.Generator().manual_seed(2147483647)
W = torch.randn((27, 2), dtype=torch.float, generator=g)
terminator = W[0]
W_lookup = W[xs].view(num_examples, 6)

assert W_lookup[0][:2].tolist() == terminator.tolist()







tensor([[ 1.5674, -0.2373,  1.5674, -0.2373,  1.5674, -0.2373],
        [ 1.5674, -0.2373,  1.5674, -0.2373, -0.4713,  0.7868],
        [ 1.5674, -0.2373, -0.4713,  0.7868,  2.4448, -0.6701],
        [-0.4713,  0.7868,  2.4448, -0.6701,  2.4448, -0.6701],
        [ 2.4448, -0.6701,  2.4448, -0.6701, -0.0274, -1.1008]])
